In [708]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from Utils import ErrorMetricsUtils as err
from Utils import CorrectnessMetricUtils as cmu
from Utils import AuxUtils as auxu
from csv import reader
import random
import time

"""
ToDo
1. What is order in norm? Are norms used correctly?
2. Optimum Learning Rate
3. General Loss Function (instead of 0-1)
4. Locally Weighted Linear Regression (if time permits)
"""

#Lasso
class l1_regularization():
    def __init__(self, alpha):
        self.alpha = alpha
    def __call__(self, W):
        return self.alpha * np.linalg.norm(W)
    def grad(self, W):
        return self.alpha * np.sign(W)
#Ridge
class l2_regularization():
    def __init__(self, alpha):
        self.alpha = alpha
    def __call__(self, W):
        return self.alpha * 0.5 *  W.T@W
    def grad(self, W):
        return self.alpha * W

In [709]:
class LinearModel():  
    
    def __init__ (self, penalty = "l1", l1_alpha = 0, l2_alpha = 0, loss = "mse", model_type = "linear", 
                  convergence = "Stochastic Gradiet Descent", include_bias = True, regression_degree = 3, 
                  standardize = True, normalize = True, learning_type = "constant", max_iter = float("inf"), 
                  learning_rate = 0.00001, epsilon = 0.0002):
        self.penalty = penalty
        self.l1_alpha = l1_alpha
        self.l2_alpha = l2_alpha
        self.loss = loss
        self.model_type = model_type
        self.convergence = convergence
        self.standardize = standardize
        self.normalize = normalize
        self.learning_type = learning_type
        self.max_iter = max_iter
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.theta = None
        self.fitted = False
        self.poly = auxu.PolynomialKernel(regression_degree, include_bias)
        if (self.model_type == "perceptron"):
            self.g = lambda x: np.piecewise(x, [x < 0, x >= 0], [0, 1])
        elif (self.model_type == "logistic"):
            self.g = lambda x: 1 / (1 + np.exp(-x))
        else:
            self.g = lambda x:x
    
    def call_convergence (self, X, y):
        if (self.convergence == "Stochatic Gradient Descent"):
            self.sch_grad_descent (X, y)
        elif (self.convergence == "Normal Equations"):
            self.normal_eqn (X, y)
        elif (self.convergence == "Newton's Method"):
             self.newton_method (X, y)
        else:
            self.bch_grad_descent (X, y)
        return
    
    # General linear model
    def fit (self, train_set):
        X_train = np.asarray(train_set)[:, :-1]
        y_train = np.asarray(train_set)[:, -1]
        if (X_train.ndim == 1):
            X_train = X_train[None, :]
        if (self.standardize):
            X_train = auxu.standardize(X_train)
        if (self.normalize):
            X_train = auxu.normalize(X_train)
        Xk_train = self.poly.kernelize(X_train)
        self.theta = np.zeros(Xk_train.shape[1])
        self.call_convergence(Xk_train, y_train)
        self.fitted = True
        return self.theta
    
    def predict (self, test_set):
        X_test = np.asarray(test_set)[:, :-1]
        if (X_test.ndim == 1):
            X_test = X_test[None, :]
        if (self.standardize):
            X_test = auxu.standardize(X_test)
        if (self.normalize):
            X_test = auxu.normalize(X_test)
        Xk_test = self.poly.kernelize(X_test)
        if (self.check_fitted(Xk_test)): 
            return self.g(Xk_test @ self.theta)
        else: 
            return -1 # some error statement
    
    def check_fitted (self, Xk_test):
        return self.fitted and self.theta.shape[0] == Xk_test.shape[1]
    
    def calc_loss (self, X, y):
        hypo = self.g (X @ self.theta)
        if (self.loss == "rmse"):
            return err.rmse_calc (y, hypo)
        elif (self.loss == "mae"):
            return err.mae_calc (y, hypo)
        elif (self.loss == "kld"):
            if (self.model_type == "logistic" or self.model_type == "perceptron"):
                return err.kl_divergence_calc (y, hypo)
            else:
                return float("Nan")
        elif (self.loss == "cross_entropy"):
            if (self.model_type == "logistic" or self.model_type == "perceptron"):
                return err.cross_entropy_calc (y, hypo)
            else:
                return float("Nan")
        else:
            # default case mse
            return err.mse_calc (y, hypo)
        
    def regularizer_neg_gradient(self):
        r_neg_g = np.zeros(self.theta.shape)
        if "l1" in self.penalty:
            l1 = l1_regularization (self.l1_alpha)
            r_neg_g = r_neg_g - l1.grad(self.theta)
        if "l2" in self.penalty:
            l2 = l2_regularization (self.l2_alpha)
            r_neg_g = r_neg_g - l2.grad(self.theta)
        return r_neg_g
    
    def neg_gradient (self, X, y):
        """
        ToDo
        General Loss (with confusion matrix)
        Does 0-1 loss work with logistic regression?
        """
        hypo = self.g (X @ self.theta)
        neg_gradient = np.zeros(self.theta.shape)
        if (self.model_type == "logistic"):
            # only for binary classification
            if (self.loss == "mse"):
                neg_gradient = (X.T)@((y-hypo)*(hypo)*(1-hypo))
            elif (self.loss == "kld" or self.loss == "cross_entropy"):
                neg_gradient = (X.T)@(y*(1-hypo))
            elif (self.loss == "mae"):
                neg_gradient = (X.T)@(np.sign(y-hypo)*(hypo)*(1-hypo))
            else:
                # default case: log loss
                neg_gradient = (X.T)@(y-hypo)
        elif (self.model_type == "perceptron"):
            # only for binary classification
            if (self.loss == "conf_mat"):
                neg_gradient = neg_gradient
            else:
                # 0-1 loss by default
                neg_gradient = (X.T)@(y-hypo)
        else:
            # Default model: linear
            if (self.loss == "mae"):
                neg_gradient = (X.T)@(np.sign(y-hypo))
            if (self.loss == "rmse"):
                rmse_err = self.calc_loss (X, y)
                neg_gradient = ((X.T)@(np.sign(y-hypo)))/rmse_err
            else: 
                # default case: mse loss
                neg_gradient = neg_gradient + (X.T)@(y-hypo)
        return neg_gradient + self.regularizer_neg_gradient()
        
    def descent_step (self, neg_gradient):
        """
        ToDo
        Optimal Leerning Rate
        """
        if (self.learning_type == "normalized"):
            beta = self.learning_rate/np.linalg.norm(neg_gradient)
        elif (self.learning_type == "constant"):
            beta = self.learning_rate
        else:
            # optimal learning type
            # ToDo
            beta = self.learning_rate
        self.theta = self.theta + beta*neg_gradient
        return np.linalg.norm(beta*neg_gradient)
        
    def print_state (self, iters, error, change_norm):
        if (iters % 100 == 0):
            print("After ", iters, " steps, the ", self.loss, " error is ", error, 
                  " and the change in theta was ", change_norm)
        return
        
    # General Batch Gradient Descent
    def bch_grad_descent (self, X, y):
        iters = 0
        while (iters < self.max_iter):
            neg_gradient = self.neg_gradient(X, y)
            change_norm = self.descent_step (neg_gradient)
            err = self.calc_loss(X, y)
            self.print_state (iters, err, change_norm)
            if (change_norm < self.epsilon): 
                break
            iters = iters + 1
        return
    
    def get_batch (self, X, y, batch_size):
        random.seed(time.time())
        index = random.randrange(X.shape[0] - batch_size)
        return X[index:index+batch_size, :], y[index:index+batch_size]
    
    # General Stochastic Gradient Descent
    def sch_grad_descent (self, X, y):
        batch_size = 49
        iters = 0
        while (iters < self.max_iter):
            X_sgd, y_sgd = self.get_batch (X, y, batch_size)
            neg_gradient = self.neg_gradient(X_sgd, y_sgd)
            change_norm = self.descent_step (neg_gradient)
            err = self.calc_loss(X, y)
            self.print_state (iters, err, change_norm)
            if (change_norm < self.epsilon): 
                break
            iters = iters + 1
        return
    
    # Newton's Method to solve regression 
    # Assumed MSE loss if linear, log loss if logistic and 0-1 loss if perceptron
    def newton_method (self, X, y):
        iters = 0;
        while (iters < self.max_iter):
            hypo = self.g (X @ self.theta)
            neg_gradient = X.T @ (y - hypo)
            hinv = np.linalg.pinv(X.T @ X)
            change = hinv @ neg_gradient
            if (np.linalg.norm (change, ord = 1) < self.epsilon): 
                break
            self.theta = self.theta + change
            iters = iters + 1
        return

    # Normal Equations to solve regression (assumed MSE loss function and linear/polynomial regression)
    def normal_eqn (self, X, y):
        self.theta = np.linalg.pinv(X.T @ X)@((X.T)@y)
        return

In [710]:
# Get CSV file
def get_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        data = reader(file)
        for row in data:
            if not row:
                continue
            dataset.append(row)
    return dataset

#String to float columnwise
def str_to_float_col(dataset, col):
    for row in dataset:
        row[col] = float(row[col].strip())

def get_csv2 (filename):
    op = np.genfromtxt(filename, delimiter=',')
    op = op[:, 1:]
    return op.astype(np.float)
        
# Split dataset into n folds
def crossval_split(dataset, n_folds):
    split = list()
    dataset_copy = list(dataset)
    fold_dim = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_dim:
            index = random.randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        split.append(fold)
    return split

# Algo evaluation by cross validation split
def eval_algo_cross_val (dataset, n_folds, *args):
    folds = crossval_split(dataset, n_folds)
    mseScores = list()
    maeScores = list()
    rmseScores = list()
    klScores = list()
    jsScores = list()
    ceScores = list()
    r2Scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = fold
        algo = LinearModel()
        algo.fit(train_set)
        predicted = algo.predict(test_set)
        actual = [row[-1] for row in fold]
        mse = err.mse_calc(actual, predicted)
        mae = err.mae_calc(actual, predicted)
        rmse = err.rmse_calc(actual, predicted)
        #kl_div = err.kl_div_calc(actual, predicted)
        #js_div = err.js_div_calc(actual, predicted)
        #cross_entropy = err.cross_entropy_calc(actual, predicted)
        r2 = err.r2_calc(actual, predicted)
        mseScores.append(mse)
        maeScores.append(mae)
        rmseScores.append(rmse)
        #klScores.append(kl_div)
        #jsScores.append(js_div)
        #ceScores.append(cross_entropy)
        r2Scores.append(r2)
    return mseScores, maeScores, rmseScores, klScores, jsScores, ceScores, r2Scores

# Train over entire set and report training Error
def eval_algo_train (dataset, n_folds, *args):
    #new_d = list ()
    #new_d.append(dataset[0])
    #new_d.append(dataset[1])
    #new_d.append(dataset[2])
    #new_d.append(dataset[3])
    #new_d.append(dataset[4])
    mseScores = list()
    maeScores = list()
    rmseScores = list()
    klScores = list()
    jsScores = list()
    ceScores = list()
    r2Scores = list()
    train_set = list(dataset)
    algo = LinearModel()
    algo.fit(train_set)
    predicted = algo.predict(train_set)
    actual = [row[-1] for row in dataset]
    mse = err.mse_calc(actual, predicted)
    mae = err.mae_calc(actual, predicted)
    rmse = err.rmse_calc(actual, predicted)
    #kl_div = err.kl_div_calc(actual, predicted)
    #js_div = err.js_div_calc(actual, predicted)
    #cross_entropy = err.cross_entropy_calc(actual, predicted)
    r2 = err.r2_calc(actual, predicted)
    mseScores.append(mse)
    maeScores.append(mae)
    rmseScores.append(rmse)
    #klScores.append(kl_div)
    #jsScores.append(js_div)
    #ceScores.append(cross_entropy)
    r2Scores.append(r2)
    #print (len(predicted), " ", len(actual))
    return mseScores, maeScores, rmseScores, klScores, jsScores, ceScores, r2Scores

# evaluate algorithm
random.seed(time.time())
filename = 'data/regression.csv'
dataset = get_csv(filename)
dataset.remove(dataset[0]) # remove headings
for i in range(len(dataset[0])): # convert dataset to float columnwise
    str_to_float_col(dataset, i) 
#print (dataset[2])
    
n_folds = 5
print('Running k-fold cross validation with 5 folds')
mseScores, maeScores, rmseScores, klScores, jsScores, ceScores, r2Scores = eval_algo_cross_val (dataset, n_folds)
print('MSE Loss: %s' % mseScores)
print('MAE Loss: %s' % mseScores)
print('RMSE Loss: %s' % rmseScores)
print('R squared Loss: %s' % r2Scores)
print('KL Divergence: %s' % klScores)
print('JS Divergence: %s' % jsScores)
print('Cross Entropy: %s' % ceScores)
print('Mean MSE Loss: %s' % (sum(mseScores)/float(len(mseScores))))
print('Mean MAE Loss: %s' % (sum(mseScores)/float(len(mseScores))))
print('Mean RMSE Loss: %s' % (sum(rmseScores)/float(len(rmseScores))))
print('Mean R squared Loss: %s' % (sum(r2Scores)/float(len(r2Scores))))
#print('Mean KL Divergence: %s' % (sum(klScores)/float(len(klScores))))
#print('Mean JS Divergence: %s' % (sum(jsScores)/float(len(jsScores))))
#print('Mean Cross Entropy: %s' % (sum(ceScores)/float(len(ceScores))))

Running k-fold cross validation with 5 folds
After  0  steps, the  mse  error is  295.5816206406569  and the change in theta was  0.13468713848893135
After  100  steps, the  mse  error is  85.80349227479924  and the change in theta was  0.056999806526034126
After  200  steps, the  mse  error is  43.39394043931435  and the change in theta was  0.029036786472486253
After  300  steps, the  mse  error is  29.808108637301714  and the change in theta was  0.01895351407038064
After  400  steps, the  mse  error is  23.020756801840225  and the change in theta was  0.014433251915564422
After  500  steps, the  mse  error is  18.812349906222117  and the change in theta was  0.0116859713268887
After  600  steps, the  mse  error is  15.97147007670486  and the change in theta was  0.009732787214775713
After  700  steps, the  mse  error is  13.960878788313458  and the change in theta was  0.008273418678826314
After  800  steps, the  mse  error is  12.481498935210192  and the change in theta was  0.007

After  7700  steps, the  mse  error is  2.880173803827662  and the change in theta was  0.0008597911795064417
After  7800  steps, the  mse  error is  2.86192236382322  and the change in theta was  0.0008492734913055671
After  7900  steps, the  mse  error is  2.8441112032338816  and the change in theta was  0.0008390483664851855
After  8000  steps, the  mse  error is  2.826722944166969  and the change in theta was  0.0008291066090857727
After  8100  steps, the  mse  error is  2.8097410171737716  and the change in theta was  0.0008194393009671856
After  8200  steps, the  mse  error is  2.7931496204686863  and the change in theta was  0.0008100377920397218
After  8300  steps, the  mse  error is  2.776933681399612  and the change in theta was  0.0008008936912731688
After  8400  steps, the  mse  error is  2.7610788200228673  and the change in theta was  0.0007919988583752717
After  8500  steps, the  mse  error is  2.745571314648588  and the change in theta was  0.0007833453960466494
After  

After  15600  steps, the  mse  error is  2.093720731260142  and the change in theta was  0.0004879442065886637
After  15700  steps, the  mse  error is  2.087793767476118  and the change in theta was  0.0004859041498224572
After  15800  steps, the  mse  error is  2.081916057402858  and the change in theta was  0.00048388904710968333
After  15900  steps, the  mse  error is  2.076086800857685  and the change in theta was  0.00048189826531721587
After  16000  steps, the  mse  error is  2.070305220017861  and the change in theta was  0.00047993119275986564
After  16100  steps, the  mse  error is  2.0645705585824436  and the change in theta was  0.0004779872384078169
After  16200  steps, the  mse  error is  2.058882080970234  and the change in theta was  0.0004760658311232637
After  16300  steps, the  mse  error is  2.0532390715521323  and the change in theta was  0.0004741664189252606
After  16400  steps, the  mse  error is  2.047640833916333  and the change in theta was  0.0004722884682818

After  24100  steps, the  mse  error is  1.7147509729823747  and the change in theta was  0.0003675313605455222
After  24200  steps, the  mse  error is  1.7113835773824724  and the change in theta was  0.00036650337975041824
After  24300  steps, the  mse  error is  1.7080349656185667  and the change in theta was  0.00036548116992531163
After  24400  steps, the  mse  error is  1.7047049804208336  and the change in theta was  0.00036446466908696904
After  24500  steps, the  mse  error is  1.7013934665139367  and the change in theta was  0.00036345381643699675
After  24600  steps, the  mse  error is  1.6981002705785304  and the change in theta was  0.0003624485523291472
After  24700  steps, the  mse  error is  1.6948252412137765  and the change in theta was  0.00036144881823771825
After  24800  steps, the  mse  error is  1.6915682289009024  and the change in theta was  0.00036045455672700376
After  24900  steps, the  mse  error is  1.6883290859677231  and the change in theta was  0.000359

After  32400  steps, the  mse  error is  1.4875630822480348  and the change in theta was  0.00029767130925687933
After  32500  steps, the  mse  error is  1.4853530882416903  and the change in theta was  0.00029698133464197633
After  32600  steps, the  mse  error is  1.4831533177120273  and the change in theta was  0.0002962942562479658
After  32700  steps, the  mse  error is  1.4809637042458086  and the change in theta was  0.0002956100548206573
After  32800  steps, the  mse  error is  1.4787841820094132  and the change in theta was  0.00029492871130361846
After  32900  steps, the  mse  error is  1.4766146857420954  and the change in theta was  0.00029425020683503594
After  33000  steps, the  mse  error is  1.4744551507493455  and the change in theta was  0.000293574522744648
After  33100  steps, the  mse  error is  1.4723055128963562  and the change in theta was  0.0002929016405507413
After  33200  steps, the  mse  error is  1.4701657086015933  and the change in theta was  0.000292231

...

After  58400  steps, the  mse  error is  1.1612212745178614  and the change in theta was  0.00016944857387155612
After  58500  steps, the  mse  error is  1.1605047420788401  and the change in theta was  0.00016914896366618373
After  58600  steps, the  mse  error is  1.1597907401764422  and the change in theta was  0.00016885013828491356
After  58700  steps, the  mse  error is  1.1590792577261146  and the change in theta was  0.00016855209498166805
After  58800  steps, the  mse  error is  1.158370283701446  and the change in theta was  0.00016825483102180528
After  58900  steps, the  mse  error is  1.1576638071338186  and the change in theta was  0.00016795834368206182
After  59000  steps, the  mse  error is  1.1569598171120576  and the change in theta was  0.0001676626302504958
After  59100  steps, the  mse  error is  1.1562583027820872  and the change in theta was  0.00016736768802643227
After  59200  steps, the  mse  error is  1.1555592533465842  and the change in theta was  0.000167

After  65800  steps, the  mse  error is  1.1143759715987338  and the change in theta was  0.0001492315282165164
After  65900  steps, the  mse  error is  1.1138201603582742  and the change in theta was  0.00014898325960741257
After  66000  steps, the  mse  error is  1.1132661961855719  and the change in theta was  0.00014873560022329022
After  66100  steps, the  mse  error is  1.1127140714872545  and the change in theta was  0.00014848854803284947
After  66200  steps, the  mse  error is  1.1121637787075556  and the change in theta was  0.00014824210101288315
After  66300  steps, the  mse  error is  1.111615310328098  and the change in theta was  0.0001479962571482408
After  66400  steps, the  mse  error is  1.1110686588676766  and the change in theta was  0.00014775101443179273
After  66500  steps, the  mse  error is  1.1105238168820508  and the change in theta was  0.0001475063708643952
After  66600  steps, the  mse  error is  1.109980776963733  and the change in theta was  0.000147262

After  73400  steps, the  mse  error is  1.0769091979392664  and the change in theta was  0.0001319663634159434
After  73500  steps, the  mse  error is  1.0764745137500709  and the change in theta was  0.0001317591328291265
After  73600  steps, the  mse  error is  1.0760411931811837  and the change in theta was  0.0001315523778519968
After  73700  steps, the  mse  error is  1.0756092309695242  and the change in theta was  0.00013134609697788345
After  73800  steps, the  mse  error is  1.0751786218765873  and the change in theta was  0.0001311402887059417
After  73900  steps, the  mse  error is  1.0747493606883132  and the change in theta was  0.00013093495154112837
After  74000  steps, the  mse  error is  1.0743214422149507  and the change in theta was  0.0001307300839941764
After  74100  steps, the  mse  error is  1.0738948612909274  and the change in theta was  0.00013052568458157102
After  74200  steps, the  mse  error is  1.0734696127747154  and the change in theta was  0.000130321

After  81000  steps, the  mse  error is  1.0474187504338002  and the change in theta was  0.00011747345054372636
After  81100  steps, the  mse  error is  1.047074271717974  and the change in theta was  0.00011729845221734627
After  81200  steps, the  mse  error is  1.046730818223128  and the change in theta was  0.00011712383000112718
After  81300  steps, the  mse  error is  1.0463883862238217  and the change in theta was  0.00011694958276821797
After  81400  steps, the  mse  error is  1.046046972011015  and the change in theta was  0.00011677570939600527
After  81500  steps, the  mse  error is  1.0457065718919774  and the change in theta was  0.00011660220876609567
After  81600  steps, the  mse  error is  1.0453671821902077  and the change in theta was  0.00011642907976429896
After  81700  steps, the  mse  error is  1.0450287992453497  and the change in theta was  0.00011625632128060963
After  81800  steps, the  mse  error is  1.0446914194131074  and the change in theta was  0.0001160

After  88500  steps, the  mse  error is  1.0241948336986826  and the change in theta was  0.0001053210535253221
After  88600  steps, the  mse  error is  1.0239179204782252  and the change in theta was  0.00010517140250655425
After  88700  steps, the  mse  error is  1.0236417933933126  and the change in theta was  0.00010502205374480464
After  88800  steps, the  mse  error is  1.0233664497427035  and the change in theta was  0.00010487300638705305
After  88900  steps, the  mse  error is  1.0230918868363745  and the change in theta was  0.00010472425958338854
After  89000  steps, the  mse  error is  1.0228181019954625  and the change in theta was  0.00010457581248699723
After  89100  steps, the  mse  error is  1.0225450925522082  and the change in theta was  0.00010442766425414894
After  89200  steps, the  mse  error is  1.0222728558499066  and the change in theta was  0.00010427981404418515
After  89300  steps, the  mse  error is  1.0220013892428472  and the change in theta was  0.00010